# Example usage of LlammaIndex for RAG


In [9]:
# Set up to use local modules
%load_ext autoreload
%autoreload 2
import os


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Gemini with LlammaIndex


In [10]:
from llama_index.llms.gemini import Gemini
from llama_index.core.llms import ChatMessage

from src import utils

import pyprojroot

import dotenv

dotenv.load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# TODO continue here

In [11]:
# Example with complete
resp = Gemini().complete("Write a poem about a magic backpack")
print(resp)

/tmp/ipykernel_82087/3917234533.py:2: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/This package will no longer be supported after version 0.6.2) -- Deprecated since version 0.6.2.
  resp = Gemini().complete("Write a poem about a magic backpack")


## The Wanderer's Pack

Old leather, worn and patched with thread,
A simple backpack, softly bred.
It doesn't boast of shining gleam,
But holds within a waking dream.

For this is no ordinary hold,
A whispered secret, centuries old.
It shifts and breathes, a gentle hum,
Whatever you need, will surely come.

Lost in the woods? A map appears,
With glowing trails to quell your fears.
Hungry and cold? A warming stew,
And blankets soft, and skies of blue.

A broken heart? A comforting rhyme,
To mend the pieces, one at a time.
A daunting task? A tool so bright,
To guide your hand and set things right.

But heed this warning, soft and low,
The pack gives freely, as you grow.
It doesn't grant a wishful plea,
But what you *need* to truly be.

It asks for kindness, courage true,
And a willingness to see things through.
So wear it lightly, with open eyes,
The Wanderer's Pack, a sweet surprise. 

For magic isn't found in things,
But in the journey that living brings.
And this old backpack, wise an

In [12]:
# Example with chat
messages = [
    ChatMessage(role="user", content="Hello friend!"),
    ChatMessage(role="assistant", content="Yarr what is shakin' matey?"),
    ChatMessage(role="user", content="Help me decide what to have for dinner."),
]
resp = Gemini().chat(messages)
print(resp)

/tmp/ipykernel_82087/3443047961.py:7: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/This package will no longer be supported after version 0.6.2) -- Deprecated since version 0.6.2.
  resp = Gemini().chat(messages)


assistant: Okay, awesome! To give you the *best* recommendation, tell me a little about what you're feeling. But here are a bunch of ideas to get us started, broken down by effort/mood:

**1. Super Easy (15 minutes or less):**

* **Pasta with Pesto:** Seriously quick and satisfying.
* **Quesadillas:** Endless filling possibilities!
* **Salad with Grilled Chicken/Fish/Tofu:** Healthy and fast.
* **Soup & Sandwich:** Classic comfort food.
* **Eggs & Toast:** Never underestimate the power of breakfast for dinner!

**2. Medium Effort (20-45 minutes):**

* **Sheet Pan Dinner:** Roast veggies and protein (chicken, sausage, salmon) all on one pan!
* **Tacos:** Always a crowd-pleaser.
* **Stir-Fry:** Use pre-cut veggies to save time.
* **Chicken/Veggie Curry:**  Can be made relatively quickly with curry paste.
* **Burgers:** Homemade or store-bought patties.

**3. More Involved (45+ minutes - for when you're feeling ambitious!):**

* **Lasagna:** A classic, but takes time.
* **Roast Chicken:**

In [13]:
# Example with streaming completion
llm = Gemini()
resp = llm.stream_complete(
    "The story of Sourcrust, the bread creature, is really interesting. It all started when..."
)

/tmp/ipykernel_82087/789428070.py:2: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/This package will no longer be supported after version 0.6.2) -- Deprecated since version 0.6.2.
  llm = Gemini()


In [16]:
try:
    for r in resp:
        print(r.text, end="")
except Exception as e:
    print(f"Error processing response chunk: {e}")

In [17]:
# Example with streaming chat
llm = Gemini(model="models/gemma-3-27b-it")
messages = [
    ChatMessage(role="user", content="Hello friend!"),
    ChatMessage(role="assistant", content="Yarr what is shakin' matey?"),
    ChatMessage(role="user", content="Help me decide what to have for dinner."),
]
resp = llm.stream_chat(messages)

/tmp/ipykernel_82087/3987730226.py:2: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/This package will no longer be supported after version 0.6.2) -- Deprecated since version 0.6.2.
  llm = Gemini(model="models/gemma-3-27b-it")


In [19]:
try:
    for r in resp:
        print(r.delta, end="")
except Exception as e:
    print(f"Error processing response chunk: {e}")

## WIP: RAG with llm=Gemini


In [20]:
from pypdf import PdfReader
from llama_index.core.schema import Document
from llama_index.core import VectorStoreIndex, ServiceContext, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

# From https://www.llamaindex.ai/blog/how-i-built-the-streamlit-llm-hackathon-winning-app-finsight-using-llamaindex-9dcf6c46d7a0


def process_pdf(pdf):
    file = PdfReader(pdf)
    text = ""
    for page in file.pages:
        text += str(page.extract_text())

    doc = Document(text=text)
    return [doc]


def get_vector_index(documents):
    llm = Gemini(model="models/gemma-3-27b-it")
    faiss_index = faiss.IndexFlatL2(d)
    vector_store = FaissVectorStore(faiss_index=faiss_index)

    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    service_context = ServiceContext.from_defaults(llm=llm)

    index = VectorStoreIndex.from_documents(
        documents, service_context=service_context, storage_context=storage_context
    )

    return index

In [21]:
pdf_path = pyprojroot.here("data/vaswani_et_al_2017.pdf")
# pdf_file = open(pdf_path, "rb")
documents = process_pdf(pdf_path)
index = get_vector_index(documents)
engine = index.as_query_engine()
query = "How has Microsoft performed in this fiscal year?"
response = engine(query)

/tmp/ipykernel_82087/4082111283.py:21: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/This package will no longer be supported after version 0.6.2) -- Deprecated since version 0.6.2.
  llm = Gemini(model="models/gemma-3-27b-it")


NameError: name 'd' is not defined

Alternative attempt
https://blog.streamlit.io/build-a-chatbot-with-custom-data-sources-powered-by-llamaindex/


In [22]:
import streamlit as st


from llama_index.core import (
    VectorStoreIndex,
    ServiceContext,
    Document,
    SimpleDirectoryReader,
)


# @st.cache_resource(show_spinner=False)
def load_data():
    llm = Gemini(model="models/gemma-3-27b-it")
    # pdf_path = pyprojroot.here("data/vaswani_et_al_2017.pdf")
    pdf_path = pyprojroot.here("data")
    reader = SimpleDirectoryReader(input_dir=pdf_path, recursive=True)
    docs = reader.load_data()
    service_context = ServiceContext.from_defaults(llm=llm)
    index = VectorStoreIndex.from_documents(docs, service_context=service_context)
    return index


index = load_data()

/tmp/ipykernel_82087/3094350285.py:14: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/This package will no longer be supported after version 0.6.2) -- Deprecated since version 0.6.2.
  llm = Gemini(model="models/gemma-3-27b-it")


ValueError: ServiceContext is deprecated. Use llama_index.settings.Settings instead, or pass in modules to local functions/methods/interfaces.
See the docs for updated usage/migration: 
https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/service_context_migration/